# Title: Autonomous Manual Construction for LLM Agents via Interactive Environmental Learning

#### Members' Names or Individual's Name:

- Yufei Ren (501311186) - yufei.ren@torontomu.ca
- Hai Son Le (501228324) - haison.le@torontomu.ca
- Divyansh Agrawal (501288889) - dagrawal@torontomu.ca


# Introduction:

#### Problem Description:

Large Language Model (LLM) agents have shown impressive capabilities in completing tasks across various domains like robotics, games, and web navigation. However, these agents often require extensive human-crafted instructions and expert prompts to adapt to new environments. This dependency limits their autonomy and adaptability, creating a significant barrier to their widespread practical deployment.

#### Context of the Problem:

As AI systems become more prevalent, their ability to autonomously adapt to new environments without extensive human guidance becomes crucial. Current approaches require significant human intervention to customize agents for specific environments, which is time-consuming, requires expertise, and doesn't scale well. Enabling LLM agents to build their own understanding through interaction would dramatically enhance their practical utility in real-world applications.

#### Limitation About other Approaches:

Existing methods either rely heavily on human-provided examples and feedback or store successful experiences as skills without extracting deeper environmental understanding, leading to the "Path Dependency" problem where agents blindly replicate previous paths without adapting to new scenarios.

#### Solution:

In this project, we instroduce AutoManual, a framework that enables LLM agents to autonomously build their understanding through interaction with the environment. The system uses two collaborative agents in an alternating process: a Planner agent that interacts with the environment using code-based plans, and a Builder agent that extracts and updates rules from these interactions. These rules are then compiled into a comprehensive instruction manual that enhances adaptability and guides even smaller LLMs to success.

# Background
- LLM for Agents Planning

<!--
| Chen et al. [4] | AutoManual builds and updates rules online through alternating planning and reflection| ALFWorld, MiniWoB++, WebArena | Current implementation places all rules directly in the context, which may not scale to very large environments -->

| Reference |Explanation |  Dataset/Input |Weakness
| --- | --- | --- | --- |
| Huang et al. [7] | CodeAsPolicy uses Python code as a policy representation for LLM agents | RoboCodeGen, HumanEval | Limited discussion on handling diverse feedback types, potentially restricting robustness in dynamic settings.
| Sun et al. [3] | AdaPlanner generates and adaptively refines code-style plans with a skill memory | ALFWorld, MiniWoB++ | Only stores successful paths without extracting general environmental knowledge

- Self-improvement of LLM Agents

| Reference |Explanation |  Dataset/Input |Weakness
| --- | --- | --- | --- |
| Yao et al. [1] | ReAct synergizes reasoning and acting in LLMs using thought-action-observation loops| ALFWorld, WebShop | Requires numerous human demonstrations and lacks mechanisms for knowledge retention
| Shinn et al. [16] | Reflexion allows LLM agents to reflect on previous failures and quickly improve plans| ALFWorld, HotPotQA | Does not extract generalizable rules, requiring repeated reflections for similar scenarios
| Wang et al. [24] | Voyager stores verified programs as skills in a library, enabling reuse for increasingly complex tasks | Minecraft-inspired tasks | Stops updating skills after storage, leading to path dependency and reduced adaptability to changing conditions


- Memory Management of LLM Agents

| Reference |Explanation |  Dataset/Input |Weakness
| --- | --- | --- | --- |
| Chen et al. [9] | CLIN updates a causal abstraction-centered memory, refining it with new trials | ScienceWorld | May struggle with scaling causal abstractions to highly complex or ambiguous environments
| Packer et al. [13] | MemGPT enables LLMs to manage working and long-term memory | Conversational tasks | Overhead of memory management may slow down real-time interactions in resource-limited settings.

- LLM for Rule Discovery

| Reference |Explanation |  Dataset/Input |Weakness
| --- | --- | --- | --- |
| Wang et al. [35] | ExpeL extracts cross-task rules from Reflexion agent trajectories, generalizing knowledge across related tasks | ALFWorld, HotPotQA | Relies on offline experiences, missing real-time updates that could enhance rule relevance
| Li et al. [2] | AutoGuide generates state-aware rules and retrieves them based on test-time states | ALFWorld, Webshop | Offline rule generation may not adapt well to dynamic or unseen environmental changes


# Methodology

Inspired by previous online reinforcement learning paradigm, AutoManual operates through a three-stage process (**Building**, **Formulating**, and **Testing**) that enables LLM agents (Planner, Builder, Consolidator, Formulator, and test-time Planner) to autonomously build their understanding of new environments. The core insight is modeling the learning of environmental rules as an optimization problem, where rules are continuously refined through interaction.

The key innovation of AutoManual is its online rule optimization approach, which continuously verifies the reliability and applicability of rules through alternating practice and management. This allows the system to address the Path Dependency problem by extracting generalizable knowledge rather than merely storing successful trajectories.

![AutoManual Methodology Diagram](./data/Method.png)

## 1. Building Stage
The Building Stage is the core of the AutoManual framework, where the system learns environmental rules by alternating between planning (interacting with the environment) and rule management (extracting and updating rules). The building stage consists of two alternating processes:


### Planner Agent:

The Planner Agent generates a free-form code-based plan as an actionable plan to complete the task and executes it in the environment and receives feedback. For each task, it produces:

- An analysis of the current situation
- A list of related rules (with IDs) relevant to the situation
- An overall plan to complete the task
- A block of Python code divided into steps with assertions

Based on the outcome (**Direct Success** (Got it right on the first try), **Indirect success** (with errors that were solved), or **Failure** (Could not complete the task)), the Planner generates a conclusion summarizing the process or reflecting on errors.

For example:
- In the case of Direct or Indirect Success, the Planner will be prompted to organize its previous code into a code block.
- For Indirect Success, it additionally summarizes the mistakes and misunderstandings that cause errors.
- For the Failure case, the Planner will be prompted to reflect on the reason for the failure carefully, suggest reasonable corrections, and specify the code segment that caused the error.

![Planner diagram](./data/Planner.png)

### Builder Agent:

Upon receiving the trajectory (thought + code) from the Planner, the Builder manages rules through a pre-defined structured rule system with four attributes:

- Rule Type (Special Phenomenon, Special Mechanism, Useful Helper Method, Success Process, Corrected Error, or Unsolved Error)
- Rule Content (description starting with applicable scenarios)
- Example (code demonstration with additional remarks)
- Validation Logs (tracking information)

Three main functions:
- write_rule: Write down a new rule with its four attributes
- update_rule: Rewrite the attributes of a existing rule
- stop_generating: When the trajectory is not needed or insufficient to derive any more new rules, the function should be called.


To ensure accurate and mitigate the risk of erroneous rule extraction, the Builder employs a case-conditioned prompting strategy that first determines whether errors stem from "Imperfect Rules" or "Imperfect Agent" and then applies appropriate prompts for rule management.

![Builder diagram](./data/Builder.png)

### Consolidator Agent:

When rules exceed the maximum number or potentially relevant or overlapped rules, this agent merges or deletes redundant rules while preserving important details with three main functions:
- get_trajectory: investigate the trajectories they depend on
- update_rule
- delete rule

## 2. Formulating Stage
The Formulator Agent categorizes the rules based on their application scenarios, drafts summarization for each category, and compiles everything into a comprehensive manual in **Markdown** format. This enhances readability and global understanding of the environment.

## 3. Testing Stage
A test-time Planner agent utilizes the generated manual to complete new tasks, demonstrating the effectiveness of the learned rules. It reads the manual's rules, applies them to the task, and writes code to act.


# Implementation

In this section, you will provide the code and its explanation. You may have to create more cells after this. (To keep the Notebook clean, do not display debugging output or thousands of print statements from hundreds of epochs. Make sure it is readable for others by reviewing it yourself carefully.)

# Demo

[Watch the demo video](./data/demo.mov)

```bash
export OPENAI_API_KEY="<your-api-key>" # a valid OpenAI API key starts with sk-
export OPENAI_BASE_URL="<your-base-url>" # e.g., https://api.openai.com/v1
```

The manuals we built are already included in each environment directory for testing. You can also build it as follows.

**ALFWorld:**

```bash
cd src/automanual_alfworld

# Building stage
python main_build.py --agent_type "autobuild_case" --run_name "autobuildcase_logs" \
			--model_name "gpt-4-1106-preview" --assistant_api --num_env_per_task 6

# Formulating stage
python main_test.py --agent_type "autobuild_case" --run_name "autobuildcase_logs" \
			--model_name "gpt-4-1106-preview" --assistant_api --mode "formulating" --is_resume

# Testing stage, 'model_name' can be replace with gpt-3.5-turbo
python main_test.py --agent_type "autobuild_case" --run_name "autobuildcase_logs" \
			--model_name "gpt-4-1106-preview" --assistant_api --mode "testing" \
			--num_env_per_task 25 --is_resume
```

`--agent_type` can be selected from "replan", "autobuild_case", "autobuild_offline".

If the building or testing process stops in the middle, you can add `--is_resume --start_env_id <stop_env_id>` to continue.

**MiniWoB++:**

```bash
cd src/automanual_miniwob

# Building stage
python main_build.py --agent_type "autobuild_case" --run_name "autobuildcase_logs" \
			--model_name "gpt-4-1106-preview" --assistant_api --num_env_per_task 6

# Formulating stage
python main_test.py --agent_type "autobuild_case" --run_name "autobuildcase_logs" \
			--model_name "gpt-4-1106-preview" --assistant_api --mode "formulating" --is_resume

# Testing stage
python main_test.py --agent_type "autobuild_case" --run_name "autobuildcase_logs" \
			--model_name "gpt-4-1106-preview" --assistant_api --mode "testing" \
			--num_env_per_task 6 --is_resume
```

- The main_build.py command runs the Building Stage, where the Planner and Builder agents collaborate to learn environmental rules through interaction, and the Consolidator organizes them. It’s like setting up a training session where a robot explores a virtual house, tries tasks, and writes down what it learns. After running this file, it will output a collection of successful code blocks (skills) from the Planner’s Direct Success or Indirect Success conclusions, stored for each task type in skill_bank.json and rule_manager.json

- The main_test.py command with 'formating' mode will summarize all rule and return Markdown format, while 'testing' mode will take that Markdown and apply to current task

# Conclusion and Future Direction
AutoManual's magic lies in its online rule optimization. Unlike older methods that save rigid “recipes” and risk getting stuck (the Path Dependency problem), AutoManual learns flexible, general rules by constantly testing and tweaking them. It's like learning to ride a bike: you wobble, adjust, and soon ride smoothly anywhere. By alternating between the Planner's actions and the Builder's reflections, rules stay up-to-date and adaptable. The manual isn't just a trophy—it's a practical tool that smaller LLMs or even humans can use to master complex environments with minimal guidance.For example, in tests on ALFWorld, AutoManual achieved a 97.4% success rate with GPT-4-turbo and 86.2% with GPT-3.5-turbo and 98.3% with GPT-4-turbo and 92.7% with GPT-3.5-turbo on MiniWoB++.

That said, the system isn't without limitations. It still heavily depends on GPT-4-turbo's capabilities, which makes us wonder how well this would work with smaller, more accessible models. According to our experiments on GPT-3.5-turbo, the testing results were not as good as using GPT-4-turbo, it takes a lot of retries to succesfully execute the action. Looking ahead, we think this work could be extended by combining AutoManual with retrieval-augmented approaches to manage larger rule sets.

# References:

[1] Yao, S., Zhao, J., Yu, D., Du, N., Shafran, I., Narasimhan, K., & Cao, Y. (2023). ReAct: Synergizing reasoning and acting in language models. International Conference on Learning Representations (ICLR).

[2] Fu, Y., Kim, D., Kim, J., Sohn, S., Logeswaran, L., Bae, K., & Lee, H. (2024). AutoGuide: Automated generation and selection of state-aware guidelines for large language model agents. arXiv preprint arXiv:2403.08978.


[3] Sun, H., Zhuang, Y., Kong, L., Dai, B., & Zhang, C. (2023). AdaPlanner: Adaptive planning from feedback with language models. Neural Information Processing Systems.

[4] Chen, M., Li, Y., Yang, Y., Yu, S., Lin, B., & He, X. (2024). AutoManual: Constructing instruction manuals by LLM agents via interactive environmental learning. Neural Information Processing Systems (NeurIPS).

[7] Liang, J., Huang, W., Xia, F., Xu, P., Hausman, K., Ichter, B., Florence, P., & Zeng, A. (2022). Code as policies: Language model programs for embodied control. 2023 IEEE International Conference on Robotics and Automation (ICRA), 9493-9500.

[9] Majumder, B. P., Dalvi, B., Jansen, P. A., Tafjord, O., Tandon, N., Zhang, L., Callison-Burch, C., & Clark, P. (2023). CLIN: A continually learning language agent for rapid task adaptation and generalization. arXiv preprint arXiv:2310.10134.

[13] Packer, C., Fang, V., Patil, S. G., Lin, K., Wooders, S., & Gonzalez, J. E. (2023). MemGPT: Towards LLMs as operating systems. arXiv preprint arXiv:2310.08560.

[16] Shinn, N., Cassano, F., Labash, B., Gopinath, A., Narasimhan, K., & Yao, S. (2023). Reflexion: Language agents with verbal reinforcement learning. Neural Information Processing Systems.

[24] Wang, G., Xie, Y., Jiang, Y., Mandlekar, A., Xiao, C., Zhu, Y., Fan, L., & Anandkumar, A. (2023). Voyager: An open-ended embodied agent with large language models. arXiv preprint arXiv:2305.16291.

[35] Zhao, A., Huang, D., Xu, Q., Lin, M., Liu, Y., & Huang, G. (2024). ExpeL: LLM agents are experiential learners. AAAI Conference on Artificial Intelligence (AAAI).